In [1]:
pip install fastapi uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.3 MB/s eta 0:00:00


In [2]:
from fastapi import FastAPI

# Create an app
app = FastAPI()

# define a path for HTTP Get method
@app.get("/")
def root():
    return {"Hello": "World"}

In [23]:
!uvicorn main:app --reload

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [1809] using StatReload
ERROR:    Error loading ASGI app. Could not import module "main".
INFO:     Stopping reloader process [1809]
^C


In [25]:
!uvicorn main:app --reload

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [2943] using StatReload
ERROR:    Error loading ASGI app. Could not import module "main".
INFO:     Stopping reloader process [2943]


In [5]:
items = []

In [7]:
@app.post("items")
def create_item(item: str):
    items.append(item)
    return item

In [9]:
!curl -X POST -H "Content-Type: application/json" 'http://127.0.0.1:8000/items?item=apple'

curl: (7) Failed to connect to 127.0.0.1 port 8000 after 0 ms: Connection refused


In [10]:
@app.get("items/{item_id}")
def get_item(item_id: int) -> str:
    item = items[item_id]
    return item

In [13]:
!curl -X GET http://127.0.0.1:8000/items/0

curl: (7) Failed to connect to 127.0.0.1 port 8000 after 0 ms: Connection refused


In [15]:
!curl -X GET http://127.0.0.1:8000/items/7

curl: (7) Failed to connect to 127.0.0.1 port 8000 after 0 ms: Connection refused


In [16]:
from fastapi import FastAPI, HTTPException

In [19]:
@app.get("items/{item_id}")
@app.get("items/{item_id}")
def get_item(item_id: int) -> str:
    if item_id < len(items):
        return items[item_id]
    else:
        raise HTTPException(status_code=404, detail=f"Item {item_id} not found")

In [22]:
!curl -X GET http://127.0.0.1:8000/items/7

curl: (7) Failed to connect to 127.0.0.1 port 8000 after 0 ms: Connection refused


In [26]:
# this endpoint uses a query parameter 'limit'
@app.get("/items/")
def list_items(limit: int = 10):
    return items[0:limit]

In [28]:
!curl -X GET 'http://127.0.0.1:8000/items?limit=3'

curl: (7) Failed to connect to 127.0.0.1 port 8000 after 0 ms: Connection refused


In [29]:
from pydantic import BaseModel

In [30]:
class Item(BaseModel):
    text: str = None
    is_done: bool = False

In [34]:
from pydantic import BaseModel

class Item(BaseModel):
    text: str = None
    is_done: bool = False

# Define the functions with proper implementation
def create_item(item: Item):
    # Add item to the list
    items.append(item)
    return item

from pydantic import BaseModel

class Item(BaseModel):
    text: str = None
    is_done: bool = False

# Define the functions with proper implementation
def create_item(item: Item):
    # Add item to the list
    items.append(item)
    return item

def get_item(item_id: int) -> Item:
    # Retrieve item from the list, handle potential IndexError
    if 0 <= item_id < len(items):
        return items[item_id]
    return None

In [36]:
!curl -X POST -H "Content-Type: application/json" 'http://127.0.0.1:8000/items?item=apple'

curl: (7) Failed to connect to 127.0.0.1 port 8000 after 0 ms: Connection refused


In [38]:
!curl -X POST -H "Content-Type: application/json" -d '{"text":"apple"}' 'http://127.0.0.1:8000/items'

curl: (7) Failed to connect to 127.0.0.1 port 8000 after 0 ms: Connection refused


In [39]:
class Item(BaseModel):
    # without default value
    text: str
    is_done: bool = False

In [41]:
!curl -X POST -H "Content-Type: application/json" -d '{"title":"apple"}' 'http://127.0.0.1:8000/items'

curl: (7) Failed to connect to 127.0.0.1 port 8000 after 0 ms: Connection refused


In [44]:
# Specify the response type will be a list of Item
@app.get("/items", response_model=list[Item])
def list_item(limit: int = 10):
    # Return the items within the specified limit
    return items[0:limit]
# Specify the response type will be a list of Item
@app.get("/items", response_model=list[Item])
def list_item(limit: int = 10):
    # Return the items within the specified limit
    return items[0:limit]
# Specify the response type will be an Item model
@app.get("/items/{item_id}", response_model=Item)
def get_item(item_id: int):
    pass